In [1]:
#### Basic Setting

import  load_dotenv
load_dotenv.load_dotenv("../../All_LLM_tutorial/.env")

True

In [2]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import chromadb
from llama_index.core.node_parser import SentenceSplitter


Settings.llm = OpenAI(model="gpt-4o-mini")
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-m3")

/home/work/enssel_test/yhkim/llamaindex_study/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
chroma_client = chromadb.PersistentClient(path="./chroma_db_example")
chroma_collection = chroma_client.get_or_create_collection("cloud")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

In [4]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)

# `pip install llama-index-llms-openai` if you don't already have it
from llama_index.llms.openai import OpenAI


class JokeEvent(Event):
    joke: str


class JokeFlow(Workflow):
    llm = OpenAI()

    @step
    async def generate_joke(self, ev: StartEvent) -> JokeEvent:
        topic = ev.topic

        prompt = f"Write your best joke about {topic}."
        response = await self.llm.acomplete(prompt)
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await self.llm.acomplete(prompt)
        return StopEvent(result=str(response))


w = JokeFlow(timeout=60, verbose=False)
result = await w.run(topic="pirates")
print(str(result))

Analysis:
This joke plays on the pun of "fish and ships" sounding like "fish and chips," a popular dish at seafood restaurants. The joke also incorporates the pirate theme by mentioning a pirate going to a seafood restaurant, which adds an element of humor.

Critique:
Overall, this joke is light-hearted and playful, making it suitable for a general audience. The pun is clever and well-executed, adding an element of surprise and humor. However, the joke may be considered somewhat predictable as the punchline is somewhat expected once the setup is established. Additionally, the humor may not be particularly sophisticated or nuanced, which could limit its appeal to certain audiences. Overall, while this joke may not be groundbreaking or particularly original, it is still a fun and enjoyable play on words.


In [11]:
from pathlib import Path

from llama_index.core.workflow import StartEvent
from llama_index.indices.managed.llama_cloud import LlamaCloudIndex
from llama_index.llms.openai import OpenAI


class MyCustomStartEvent(StartEvent):
    a_string_field: str
    a_path_to_somewhere: Path
    an_index: VectorStoreIndex
    an_llm: OpenAI

In [19]:
class JokeFlow(Workflow):
    ...

    @step
    async def generate_joke_from_index(
        self, ev: MyCustomStartEvent
    ) -> JokeEvent:
        # Build a query engine using the index and the llm from the start event
        query_engine = ev.an_index.as_query_engine(llm=ev.an_llm)
        topic = query_engine.query(
            f"What is the closest topic to {ev.a_string_field}"
        )
        # Use the llm attached to the start event to instruct the model
        prompt = f"Write your best joke about {topic}."
        response = await ev.an_llm.acomplete(prompt)
        # Dump the response on disk using the Path object from the event
        ev.a_path_to_somewhere.write_text(str(response))
        # Finally, pass the JokeEvent along
        print(str(response))
        return JokeEvent(joke=str(response))

    @step
    async def critique_joke(self, ev: JokeEvent) -> StopEvent:
        joke = ev.joke

        prompt = f"Give a thorough analysis and critique of the following joke: {joke}"
        response = await Settings.llm.acomplete(prompt)
        return StopEvent(result=str(response))

In [20]:
custom_start_event = MyCustomStartEvent(
    a_string_field="gumba",
    a_path_to_somewhere=Path("joke.txt"),
    an_index=index,
    an_llm=Settings.llm,
)
w = JokeFlow(timeout=60, verbose=False)
result = await w.run(start_event=custom_start_event)
print(str(result))

Why did the computer go to therapy?

Because it had too many unresolved issues!
This joke employs a classic structure of humor that relies on wordplay and the anthropomorphism of technology. Let's break it down into its components for a thorough analysis and critique.

### Structure and Setup

1. **Setup**: "Why did the computer go to therapy?"
   - This line sets up an expectation. It introduces a scenario that is inherently humorous because it anthropomorphizes a computer, attributing human-like emotions and behaviors (such as seeking therapy) to an inanimate object. This creates a cognitive dissonance that is often a source of humor.

2. **Punchline**: "Because it had too many unresolved issues!"
   - The punchline delivers the humor through a play on words. The term "unresolved issues" can refer to both psychological problems that a person might bring to therapy and technical problems that a computer might have, such as bugs or errors in its programming.

### Analysis of Humor Elem

In [30]:
from llama_index.utils.workflow import (
    draw_all_possible_flows,
    draw_most_recent_execution,
)

# Draw all
# draw_all_possible_flows(JokeFlow, filename="joke_flow_all.html")

# Draw an execution
w = JokeFlow()
await w.run(start_event=custom_start_event)
draw_most_recent_execution(w, filename="joke_flow_recent.html")

Why did the computer go to therapy?

Because it had too many bytes of unresolved issues!
joke_flow_recent.html


# Global management

In [31]:
from llama_index.core.workflow import Context


@step
async def query(self, ctx: Context, ev: MyEvent) -> StopEvent:
    # retrieve from context
    query = await ctx.store.get("query")

    # do something with context and event
    val = ...
    result = ...

    # store in context
    await ctx.store.set("key", val)

    return StopEvent(result=result)

NameError: name 'MyEvent' is not defined

In [36]:
from pydantic import BaseModel, Field, field_validator, field_serializer
from typing import Union
from pydantic.config import ConfigDict   # v2

# This is a random object that we want to use in our state
class MyRandomObject:
    def __init__(self, name: str = "default"):
        self.name = name


# This is our state model
# NOTE: all fields must have defaults
class MyState(BaseModel):
    my_obj: MyRandomObject = Field(default_factory=MyRandomObject)
    some_key: str = Field(default="some_value")

    model_config = ConfigDict(
        arbitrary_types_allowed=True     # 👉 커스텀 타입 허용
    )

    # # This is optional, but can be useful if you want to control the serialization of your state!

    # @field_serializer("my_obj", when_used="always")
    # def serialize_my_obj(self, my_obj: MyRandomObject) -> str:
    #     return my_obj.name

    # @field_validator("my_obj", mode="before")
    # @classmethod
    # def deserialize_my_obj(
    #     cls, v: Union[str, MyRandomObject]
    # ) -> MyRandomObject:
    #     if isinstance(v, MyRandomObject):
    #         return v
    #     if isinstance(v, str):
    #         return MyRandomObject(v)

    #     raise ValueError(f"Invalid type for my_obj: {type(v)}")

In [38]:
from llama_index.core.workflow import (
    Context,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)


class MyWorkflow(Workflow):
    @step
    async def start(self, ctx: Context[MyState], ev: StartEvent) -> StopEvent:
        # Returns MyState directly
        state = await ctx.store.get_state()
        state.my_obj.name = "new_name"
        await ctx.store.set_state(state)

        # Can also access fields directly if needed
        name = await ctx.store.get("my_obj.name")
        await ctx.store.set("my_obj.name", "newer_name")

        return StopEvent(result="Done!")

In [41]:
from llama_index.core.workflow import step, Context, Event, Workflow


class MyEvent(Event):
    pass


class MyEventResult(Event):
    result: str


class GatherEvent(Event):
    pass


class MyWorkflow(Workflow):
    @step
    async def dispatch_step(
        self, ctx: Context, ev: StartEvent
    ) -> MyEvent | GatherEvent:
        ctx.send_event(MyEvent())
        ctx.send_event(MyEvent())

        return GatherEvent()

    @step
    async def handle_my_event(self, ev: MyEvent) -> MyEventResult:
        return MyEventResult(result="result")

    @step
    async def gather(
        self, ctx: Context, ev: GatherEvent | MyEventResult
    ) -> StopEvent | None:
        # wait for events to finish
        events = ctx.collect_events(ev, [MyEventResult, MyEventResult])
        if not events:
            return None

        return StopEvent(result=events)

In [ ]:
w = MyWorkflow(...)

handler = w.run(topic="Pirates")

async for event in handler.stream_events():
    print(event)

result = await handler